In [ ]:
import geopy
import os


with open(os.path.expanduser('~/.secret/smarty_streets.json')) as f:
  keys = json.load(f)



address = '16613 SE 16th St, Bellevue'

In [ ]:
# Make an intereactive predictor

# import ipywidgets as widgets
# from ipywidgets import interact, interact_manual

# def predict_price(coefs, column1_value, column2_value):
#     display(df.loc[(df['views'] < column1_value) & 
#                     (df['reads'] < column2_value), 
#                    ['title', 'read_time', 'tags', 'views', 'reads']])
        
# column1_value=widgets.IntText(value=100, label='First')
# column2_value=widgets.IntSlider(value=100, label='Second')

# linked = widgets.jslink((column1_value, 'value'),
#                         (column2_value, 'value'))

# less_than = interact(show_less_than, column1_value=column1_value,
#                  column2_value=column2_value)

In [1]:
# GEOCODING

from geopy import Nominatim
import time

ModuleNotFoundError: No module named 'geopy'

In [ ]:
# Find school district
import folium  # for plotting stuff later if needed.   Would be fun to do.
import json

# might not need all of these
from shapely.geometry import Polygon, Point, MultiPolygon, shape, GeometryCollection



my_shapes = "districts.geojson"  # geojson shape file for district boundaries

with open(my_shapes, 'r') as f:
    school_geo_data = json.load(f)


# make a list of the names of the school districts to appy later
dist_names = [x["properties"]["NAME"] for x in school_geo_data['features']]  # It worked!

# NOTE: buffer(0) is a trick for fixing scenarios where polygons have overlapping coordinates 
# We go through every item in the geojson features, and create mappable polygons from it
polygons = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in school_geo_data['features']])
polygons[0]


def make_point(long, lat):
    '''
    Returns a shapely Point object from a latlong location.

            Parameters:
                    latlong (list): iterable of len 2 [lat, long]

            Returns:
                    Point object
    '''
    # note that the point requires (long, lat) or (x, y) format
    return Point(long, lat)


def which_district(point, polygon_list, district_names):
    '''
    Returns the name from district_names list if point is in polygon from polygon_list
            
            Parameters:
                    point (Point object): Shapely Point
                    polygon_list (list): Shapely Shape objects (MultiPolygon)
                    district_names (list): list of school district names taken from geo_data

            Returns:
                    binary_sum (str): Binary string of the sum of a and b
    '''
    count = 0
    name = 'None'
    for i in range(len(polygon_list)):
        if polygon_list[i].contains(point):
            name = district_names[i]
            count += 1
    if count == 1: return name
    else: print("District ID failure at", point)


def find_closest_water(point, df):
    # return the minimum distance from a waterfront property in miles
    all_locs = df['points']
    closest = all_locs.apply(point.distance).min()
    miles = closest * 68.94  # convert degrees to miles
    miles += 0.1 # keeps us from getting zeros in my data (prevents divide by zero later)
    return miles



In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual



dist_scores = [ ['Seattle', 4],
                ['Federal Way', 0],  
                ['Enumclaw', 1],
                ['Mercer Island', 6],
                ['Highline', 0],
                ['Vashon Island', 4],
                ['Renton', 1],
                ['Skykomish', 1],
                ['Bellevue', 6],
                ['Tukwila', 1],
                ['Riverview', 3],
                ['Auburn', 2],
                ['Tahoma', 5],
                ['Snoqualmie Valley', 3],
                ['Issaquah', 6],
                ['Shoreline', 5],
                ['Lake Washington', 6],
                ['Kent', 2],
                ['Northshore', 5],
                ['Fife', 2],
                ]


class Calculator():
    def __init__(self, regression, data, polygons):
        self.district = "Seattle"
        self.dist_score = 4
        self.regression = regression
        self.sqft = 2000
        self.grade = 7
        self.bedrooms = 3
        self.baths = 2
        self.geolocator = Nominatim(user_agent='my_app')
        self.lat = 47.6062
        self.long = -122.3321
        self.point = Point(self.long, self.lat)
        self.address = '3244 139th Ave SE, Bellevue, WA 98005'
        self.dist_scores = [ ['Seattle', 4],
                ['Federal Way', 0],  
                ['Enumclaw', 1],
                ['Mercer Island', 6],
                ['Highline', 0],
                ['Vashon Island', 4],
                ['Renton', 1],
                ['Skykomish', 1],
                ['Bellevue', 6],
                ['Tukwila', 1],
                ['Riverview', 3],
                ['Auburn', 2],
                ['Tahoma', 5],
                ['Snoqualmie Valley', 3],
                ['Issaquah', 6],
                ['Shoreline', 5],
                ['Lake Washington', 6],
                ['Kent', 2],
                ['Northshore', 5],
                ['Fife', 2],
                ]
        self.dist_names = [x[0] for x in self.dist_scores]
        self.nearest_water = 0.2
        self.data = data
        self.polygons = polygons
        self.price = 0
        self.cols = ['sqft_living', 'nearest_water', 'bedrooms', 'bathrooms', 'grade',
       'school_district_Bellevue', 'school_district_Enumclaw',
       'school_district_Federal Way', 'school_district_Fife',
       'school_district_Highline', 'school_district_Issaquah',
       'school_district_Kent', 'school_district_Lake Washington',
       'school_district_Mercer Island', 'school_district_Northshore',
       'school_district_Renton', 'school_district_Riverview',
       'school_district_Seattle', 'school_district_Shoreline',
       'school_district_Skykomish', 'school_district_Snoqualmie Valley',
       'school_district_Tahoma', 'school_district_Tukwila',
       'school_district_Vashon Island']

    def get_school_score(self, dist_scores):
        for name, score in self.dist_scores:
            if self.district == name:
                return score
            
    def get_district(self):
        self.district = which_district(self.point, self.polygons, self.dist_names)
    
    def change_dist(self, school_district):
        self.dist_score = school_district

    def change_sqft(self, sqft):
        self.sqft = sqft
    
    def change_grade(self, grade):
        self.grade = grade
    
    def change_bedrooms(self, bedrooms):
        self.bedrooms = bedrooms
    
    def change_baths(self, baths):
        self.baths = baths
        
    def change_address(self, address):
        self.address = address
        self.long, self.lat = self.get_long_lat()
        self.point = make_point(self.long, self.lat)
        print(address)
        print("Location:", self.lat, self.long)
        print("School District:", self.district, "(score = " + str(self.dist_score) + ")")
        self.nearest_water = find_closest_water(self.point, self.data[self.data['nearest_water'] <= 0.11])
        print("Nearest Waterfront:", self.nearest_water)
       
    def get_long_lat(self):
        time.sleep(1)
        location = self.geolocator.geocode(self.address)
        return location.longitude, location.latitude
    
    def build_X(self):
        dist_col = 'school_district_' + self.district        
        x = self.cols[:]
        x[0] = self.sqft
        x[1] = self.nearest_water
        x[2] = self.bedrooms
        x[3] = self.baths
        x[4] = self.grade
           
        X = []
        for col in x:
            if col == dist_col:
                X.append(1)
            elif type(col) is type('str'):
                X.append(0)
            else:
                X.append(col)
        
        return X
        
    
    def predict(self, clicked):
        X = self.build_X()
        X = pd.DataFrame([X], columns=self.cols)
        print(X.head())
        self.price = self.regression.predict(X)[0]
        print("Predicted Sale Price: ${:,.2f}".format(self.price))
    
    
print("HOME VALUE CALCULATOR")
calc = Calculator(regression, kc2, polygons)
interact(calc.change_sqft, sqft=widgets.IntSlider(min=1500, max=5000, step=25, value=calc.sqft));
interact(calc.change_dist, school_district=dist_scores);
interact(calc.change_grade, grade=widgets.IntSlider(min=1, max=13, step=1, value=calc.grade));
interact(calc.change_bedrooms, bedrooms=widgets.IntSlider(min=1, max=6, step=1, value=calc.bedrooms));
interact(calc.change_baths, baths=widgets.FloatSlider(min=1, max=5, step=0.25, value=calc.baths));
interact(calc.change_address, address=widgets.Text(
    value=calc.address,
    placeholder='Enter Address',
    description='Address:',
    disabled=False,
    continuous_update=False
))


button = widgets.Button(description="Calculate!")
output = widgets.Output()

display(button, output)


button.on_click(calc.predict)